# YOUR PROJECT TITLE

Imports and set magics:

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from matplotlib_venn import venn2 # install with pip install matplotlib-venn

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
import dataproject

# Read and clean data

In [4]:
import requests

In [19]:
response = requests.get("https://api.statbank.dk/v1/data?format=JSON")
print(response.status_code)

400


In [18]:
reponse = requests.get("https://api.statbank.dk/v1/subjects?format=JSON")
print(response.status_code)

404


In [ ]:
#example

#Specifying query parameters: 
#Personal API-key to access data, data period by using our date_time function, stationId (06184 is the DMI station at Østerbro, Kbh), 
#parameterId (temp_dry is "Present air temperature measured 2 m over terrain every 10 min") and finally the limit
#which should be around 6*24*(3650+240)=560160 observations - thus the limit should be more than this (incl. 27 of august)
params = {'api-key' : api_key,
          'from' : date_time(start_time),
          'to' : date_time(end_time),
          'stationId' : '06184',
          'parameterId' : 'temp_dry',
          'limit' : '1000000',
          }
#Requests and gets URL for the chosen query parameters.
r = requests.get(url, params=params)
print(r, r.url)

In [14]:
# https://api.statbank.dk 
# https://api.statbank.dk/v1/data

# Merge data sets

# Data Analysis


# Conclusion

ADD CONCISE CONLUSION.